**Pomembni stolpci**

In [13]:
import pandas as pd

df = pd.read_csv('koncniPodatkiZaModel.csv', sep=',')

pomembni = df[['smart_5_raw','smart_187_raw','smart_188_raw']]
display(pomembni)
display(pomembni.isnull().sum())

# !!odpovedi je samo 8, rabimo utezi, vec takih primerov iz drugih datasetov...
odpovedi = df.loc[df['failure'] == 1]
display(odpovedi)

,smart_5_raw,smart_187_raw,smart_188_raw
0,0.0,NaN,NaN
1,NaN,NaN,NaN
2,2657.0,NaN,NaN
3,144.0,683.0,4.295361e+09
4,843.0,NaN,NaN
...,...,...,...
8823,0.0,NaN,NaN
8824,0.0,NaN,NaN
8825,0.0,NaN,NaN
8826,0.0,NaN,NaN


smart_5_raw        65
smart_187_raw    5555
smart_188_raw    5563
dtype: int64

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
1,2025-04-27,D7JDTYHN,HGST HUH721212ALE604,12000138625024,1,phx1,0,1059,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-04,QGK9NWST,WDC WUH721414ALE6L4,14000519643136,1,sac2,0,1152,18,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-02-28,ZHZ3CQL6,ST14000NM0138,14000519643136,1,sac0,0,1154,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-14,1180A03AF97G,TOSHIBA MG07ACA14TA,14000519643136,1,sac2,0,1160,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-11-14,6250A0N8FVKG,TOSHIBA MG08ACA16TA,16000900661248,1,iad1,50,1004,5,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,2025-09-24,81K0A0BAFV8G,TOSHIBA MG08ACA16TEY,16000900661248,1,ams5,31,2017,19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8820,2024-10-14,ZL2LN4XM,ST16000NM001G,16000900661248,1,phx1,40,2007,16,42.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8821,2025-09-16,X880A0HBF97G,TOSHIBA MG07ACA14TA,14000519643136,1,phx1,0,1063,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8823,2025-05-28,8CH0VY3E,HGST HUH721212ALN604,12000138625024,1,sac2,0,1126,12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Rough Preprocessing**

odstranjevanje vecine NaN vrednosti, fokus na pomembne stolpce, ....

In [15]:
import pandas as pd

df = pd.read_csv('koncniPodatkiZaModel.csv', sep=',')
display(df.head(5))
display(df.shape)

NaN = df.isnull().mean()

nujni_stolpci = [
    'smart_5_raw', 
    'smart_187_raw', 
    'smart_188_raw', 
    'smart_197_raw', 
    'smart_198_raw', 
    'smart_9_raw', 
    'smart_194_raw', 
    'capacity_bytes', 
    'model',
    'failure'
]

#obdrži stolpec, če ima < 30% NaN ALI če je na seznamu nujnih
relevantni = NaN[(NaN < 0.3) | (NaN.index.isin(nujni_stolpci))].index.tolist()

print(f"Obdržali bomo {len(relevantni)} stolpcev.")

zaStran = [col for col in df.columns if col not in relevantni]
df = df.drop(columns=zaStran)

display(df.shape)
display(df)

# "lokacijski stolpci" od blackblaze fizicnih serverjev, ki jih ne rabimo
lokacijski = df[['datacenter','cluster_id','vault_id','pod_id','pod_slot_num','is_legacy_format']]

#ostali nepomembni
nepomembniOstalo = df[['date','serial_number']]

# vsi "normalized" so samo interpetacija proizvajalca, ker je vec proizvajalcev so nekonsistentni
def vsiNorm(df):
    normStolpci = []

    for col in df.columns:
        if 'normalized' in col:
            normStolpci.append(col)
    return normStolpci

normStolpci = vsiNorm(df)
df = df.drop(columns=normStolpci)
df = df.drop(columns=lokacijski)
df = df.drop(columns=nepomembniOstalo)
display(df.shape)   #19 dejansko uporabnih stolpcev ..
display(df.columns)
print(df.head())

#df.to_csv('izbolsani_podatki.csv', index=False)

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2025-04-08,1ZG0WX3D,WDC WUH722222ALE6L4,22000969973760,0,iad1,50,1021,0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-04-27,D7JDTYHN,HGST HUH721212ALE604,12000138625024,1,phx1,0,1059,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-04,QGK9NWST,WDC WUH721414ALE6L4,14000519643136,1,sac2,0,1152,18,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-02-28,ZHZ3CQL6,ST14000NM0138,14000519643136,1,sac0,0,1154,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-14,1180A03AF97G,TOSHIBA MG07ACA14TA,14000519643136,1,sac2,0,1160,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(8828, 197)

Obdržali bomo 40 stolpcev.


(8828, 40)

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,is_legacy_format,...,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw
0,2025-04-08,1ZG0WX3D,WDC WUH722222ALE6L4,22000969973760,0,iad1,50,1021,0,False,...,100.0,540.0,57.0,38.0,100.0,0.0,100.0,0.0,100.0,0.0
1,2025-04-27,D7JDTYHN,HGST HUH721212ALE604,12000138625024,1,phx1,0,1059,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-04,QGK9NWST,WDC WUH721414ALE6L4,14000519643136,1,sac2,0,1152,18,False,...,99.0,7679.0,65.0,30.0,100.0,0.0,100.0,0.0,100.0,0.0
3,2025-02-28,ZHZ3CQL6,ST14000NM0138,14000519643136,1,sac0,0,1154,3,False,...,99.0,2597.0,29.0,29.0,94.0,3040.0,94.0,3040.0,200.0,0.0
4,2025-01-14,1180A03AF97G,TOSHIBA MG07ACA14TA,14000519643136,1,sac2,0,1160,16,False,...,100.0,113.0,100.0,33.0,100.0,5.0,100.0,0.0,200.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8823,2025-05-28,8CH0VY3E,HGST HUH721212ALN604,12000138625024,1,sac2,0,1126,12,False,...,99.0,2023.0,166.0,36.0,100.0,1711.0,100.0,0.0,200.0,0.0
8824,2025-01-19,70X0A03QF97G,TOSHIBA MG07ACA14TA,14000519643136,0,sac2,0,1159,16,False,...,100.0,138.0,100.0,32.0,100.0,0.0,100.0,0.0,200.0,0.0
8825,2025-07-31,8160A01YFVKG,TOSHIBA MG08ACA16TA,16000900661248,0,phx1,0,1415,9,False,...,100.0,17.0,100.0,38.0,100.0,0.0,100.0,0.0,200.0,0.0
8826,2025-05-07,41D0A0FUFV8G,TOSHIBA MG08ACA16TEY,16000900661248,1,ams5,31,2008,14,False,...,100.0,70.0,100.0,34.0,NaN,NaN,100.0,0.0,200.0,0.0


KeyError: "['pod_slot_num'] not found in axis"

**Vrte diskov**

In [ ]:
unikatneVrsteDiskov = df['model'].unique()
print("vrste diskov: ", unikatneVrsteDiskov)

ssd_keywords = ['SSD', 'MTFD', 'SSDSC', '850 PRO', '870 EVO', '860 PRO', '5300']
df['jeSSD'] = df['model'].apply(lambda x: 1 if any(k in str(x).upper() for k in ssd_keywords) else 0)

def getModel(model):
    m = str(model).upper()
    if m.startswith('ST') or 'SEAGATE' in m: return 'Seagate'
    if m.startswith('WDC') or m.startswith('WD') or 'WESTERN' in m: return 'Western Digital'
    if m.startswith('HGST') or m.startswith('HUH') or m.startswith('HMS'): return 'HGST'
    if m.startswith('TOSHIBA') or m.startswith('MG'): return 'Toshiba'
    if m.startswith('SAMSUNG'): return 'Samsung'
    if m.startswith('CT') or 'CRUCIAL' in m: return 'Crucial'
    return 'Other'

df['model'] = df['model'].apply(getModel)

display(df)
ostali = df[df['model'] == 'Other']['model'].unique()
print("nekategorizirani modeli: ", ostali)
modeli = df['model'].unique()
display("kategorizirani modeli: ", modeli)

**Temeljit Preprocessing**

POMEMBNO - Zapolni NaN v teh nujnih stolpcih z 0
Ker XGBoost ne mara popolnih praznin, manjše število NaN pa zna rešiti, 

In [ ]:
display(df)

NaNbefore = df.isnull().sum()
print("trenutno Nan: ", NaNbefore)

#pretvorimo byte v gigabyte
df = df.rename(columns={"capacity_bytes":"capacity_gigabytes"})
df['capacity_gigabytes'] = (df['capacity_gigabytes'] / (1024 ** 3)).round(2)
display(df)

#VSE NaN VREDNOSTI

#ŠTEVCI NAPAK (Error Counters)
#tukaj je "0" kul, ker predpostavimo, da če ni podatka, ni bilo napak
#to velja tako za SSD kot za HDD

stevcneNapake = ['smart_1_raw', 'smart_7_raw', 'smart_10_raw', 'smart_192_raw', 'smart_199_raw']
df[stevcneNapake] = df[stevcneNapake].fillna(0)

#MEHANSKI CIKLI (Mechanical Cycles) pazi: SSD vs HDD
#za SSD (jeSSD == 1) damo 0, ker nima motorja in glav
#za HDD (jeSSD == 0) pa ne smemo dati 0, zato uporabimo mediano (srednjo vrednost)
mehanskiStolpci = ['smart_3_raw', 'smart_4_raw', 'smart_193_raw']

for col in mehanskiStolpci:
    # 1. Za SSD varno zapolnimo z 0
    df.loc[(df['jeSSD'] == 1) & (df[col].isnull()), col] = 0
    
    # 2. Za HDD izračunamo mediano (da ne uničimo podatkov z 0)
    mediana_hdd = df.loc[df['jeSSD'] == 0, col].median()
    df.loc[(df['jeSSD'] == 0) & (df[col].isnull()), col] = mediana_hdd

#SPLOŠNA PORABA (Power Cycle)
#vsak disk se vklopi, zato tukaj uporabimo povprečje, ker 0 ni realna.
df['smart_12_raw'] = df['smart_12_raw'].fillna(df['smart_12_raw'].median())

#TEMPERATURA, ITD..
nujniOstali = ['smart_5_raw', 'smart_9_raw', 'smart_187_raw', 'smart_188_raw', 'smart_194_raw', 'smart_197_raw', 'smart_198_raw']
df[nujniOstali] = df[nujniOstali].fillna(0)

NaNafter = df.isnull().sum()
print("NaN po procesiranju ", NaNafter)
print(df.head())

#df.to_csv('izbolsani_podatki_sample.csv', index=False)